In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Libraries
import os
import pickle

from src.models import build_model, train_model, test_model, predict_model

In [3]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [4]:
EXPERIMENT = 'mh01'
MODEL = 'modelA'
EXP_DIR = f'./models/{EXPERIMENT}/'
# Training
PATH_TO_DATA = f'data/processed/synthetic_data/{MODEL}/'
init_occupancy = 'expected'
# Models
model_template = f'./models/templates/{MODEL}.pl'
model = f'./models/{EXPERIMENT}/{MODEL}.pl'

### Training

Add prior knowledge to model template

In [5]:
pk_dict = train_model.get_pk(PATH_TO_DATA)
build_model.add_pk(model_template, model, pk_dict)

Train model

In [7]:
train_model.main(
    experiment = EXPERIMENT, 
    path_to_data = f'{PATH_TO_DATA}n1000/', 
    model_template = model, 
    training = 0.8, 
    max_iter = 3, 
    evidence = ['activity', 'occupancy', 'fold_change'],
    facts = [],
    init_occupancy = init_occupancy,
    suffix = ''
)

Splitting data...
Writing data to files...
Writing evidence to file...
Starting LFI...
problog lfi ./models/mh01/modelA.pl ./models/mh01/evidence.pl -O ./models/mh01/model_trained.pl --normalize -k sdd -v -n 3
[INFO] Output level: INFO
[INFO] Number of examples: 800
[INFO] Compile time: 177.496655
[INFO] Weights to learn: [t(0.33,E)::activity(E,S,inc), t(0.33,E)::activity(E,S,dec), t(0.33,E)::activity(E,S,baseline), t(0.33,P)::fold_change(P,S,inc), t(0.33,P)::fold_change(P,S,dec), t(0.33,P)::fold_change(P,S,baseline), t(0.33,E,_dM)::occupancy(P,S,inc), t(0.33,E,_dM)::occupancy(P,S,dec), t(0.33,E,_dM)::occupancy(P,S,baseline), t(0.33,_fc)::occupancy(P,S,inc), t(0.33,_fc)::occupancy(P,S,dec), t(0.33,_fc)::occupancy(P,S,baseline)]
[INFO] Bodies: [lfi_body(0,t(E),E,S,inc), lfi_body(1,t(E),E,S,dec), lfi_body(2,t(E),E,S,baseline), lfi_body(3,t(P),P,S,inc), lfi_body(4,t(P),P,S,dec), lfi_body(5,t(P),P,S,baseline), lfi_body(6,t(E,_dM),P,S,inc), lfi_body(7,t(E,_dM),P,S,dec), lfi_body(8,t(E,_dM),

Testing

In [17]:
# Testing
model_trained = f'{EXP_DIR}model_trained.pl'
test = ['training', 'testing']
PATH_TO_TRUTH = f'data/processed/synthetic_data/{MODEL}/n1000/'
results = {}

In [ ]:
for test_on in test:
    PATH_TO_TESTING = f'{EXP_DIR}{test_on}/'
    compare_to = [test_on, 'truth']
    for compare in compare_to:
        if compare == 'truth':
            path_to_truth = PATH_TO_TRUTH
        else: 
            path_to_truth = PATH_TO_TESTING

        test_results = test_model.main(
            experiment = EXPERIMENT, 
            path_to_data = PATH_TO_TESTING, 
            path_to_truth = path_to_truth, 
            model_trained = model_trained, 
            targets = ['occupancy', 'modification'], 
            test_on = test_on, compare = compare)
        results[f'{test_on}_vs_{compare}'] = test_results

In [19]:
PATH_TO_TESTING = f'{EXP_DIR}testing/'
predictions = predict_model.main(
    experiment = EXPERIMENT, 
    path_to_data = PATH_TO_TESTING, 
    model_trained = model_trained, 
    targets = ['occupancy', 'modification'], 
    mode = 'inference', infer_from = 'facts', 
    suffix = '_testing')
results['predictions'] = predictions

Inferring data (s10)...
problog ./models/mh01/make_predictions.pl -o ./models/mh01/predictions/inferred_example.txt
Finished inference (s10)...
Inferring data (s100)...
problog ./models/mh01/make_predictions.pl -o ./models/mh01/predictions/inferred_example.txt
Finished inference (s100)...
Inferring data (s101)...
problog ./models/mh01/make_predictions.pl -o ./models/mh01/predictions/inferred_example.txt
Finished inference (s101)...
Inferring data (s109)...
problog ./models/mh01/make_predictions.pl -o ./models/mh01/predictions/inferred_example.txt
Finished inference (s109)...
Inferring data (s117)...
problog ./models/mh01/make_predictions.pl -o ./models/mh01/predictions/inferred_example.txt
Finished inference (s117)...
Inferring data (s12)...
problog ./models/mh01/make_predictions.pl -o ./models/mh01/predictions/inferred_example.txt
Finished inference (s12)...
Inferring data (s126)...
problog ./models/mh01/make_predictions.pl -o ./models/mh01/predictions/inferred_example.txt
Finished in

In [20]:
with open(f'{EXP_DIR}results.pickle', 'wb') as file:
    pickle.dump(results, file, protocol=pickle.HIGHEST_PROTOCOL)